<a href="https://colab.research.google.com/github/xxKeyaxx/Multiple-Floor-Indoor-Positioning-System-using-GraphSage-and-Transfer-Learning/blob/main/Indoor_Positioning_System_Skripsi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
!pip install torch-geometric

In [67]:
from google.colab import drive
drive.mount('/content/drive')

import torch
import pandas as pd
import numpy as np
from torch_geometric.data import Data, Batch
from torch_geometric.nn import SAGEConv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

test_file_path = '/content/drive/My Drive/UTSIndoorLoc/UTS_test.csv'
train_file_path = '/content/drive/My Drive/UTSIndoorLoc/UTS_training.csv'
df_test = pd.read_csv(test_file_path)
df_train = pd.read_csv(train_file_path)
print(df_train.head())  # Preview the data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   WAP001  WAP002  WAP003  WAP004  WAP005  WAP006  WAP007  WAP008  WAP009  \
0     100     100     100     100     100     100     100     100     100   
1     100     100     100     100     100     100     100     100     100   
2     100     100     100     100     100     100     100     100     100   
3     100     100     100     100     100     100     100     100     100   
4     100     100     100     100     100     100     100     100     100   

   WAP010  ...  WAP587  WAP588  WAP589    Pos_x    Pos_y  Floor_ID  \
0     100  ...     100     100     100  40.7871  31.0272         8   
1     100  ...     100     100     100  40.7871  31.0272         8   
2     100  ...     100     100     100  40.7871  31.0272         8   
3     100  ...     100     100     100  39.4464  30.7985         8   
4     100  ...     100     100     100  39.4464  30.7985  

In [68]:
# Assuming data is in a Pandas DataFrame
floor_datasets = {}
for floor_id in df_train["Floor_ID"].unique():
    floor_datasets[floor_id] = df_train[df_train["Floor_ID"] == floor_id]

In [69]:
def preprocess_data(data):
    # Replace 100 with 0 (no detection)
    sensor_cols = [col for col in data.columns if col.startswith('WAP')]
    # data[sensor_cols] = data[sensor_cols].replace(100, 0)
    return data, sensor_cols

df_train, sensor_cols = preprocess_data(df_train)
# df_train = df_train[df_train['Floor_ID'] == 12]

# Load and preprocess the new dataset
df_test, sensor_cols = preprocess_data(df_test)
# max_floor = 12
# df_test = df_test[df_test['Floor_ID'] == max_floor]

SVM

In [70]:
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def data_split(data):
    x = data[sensor_cols].values
    y = data[['Pos_x', 'Pos_y', 'Floor_ID']].values
    return x, y

x_train, y_train = data_split(df_train)
x_test, y_test = data_split(df_test)

# Initialize the SVR model
svr_model = SVR(kernel='rbf', C=1.0, epsilon=0.2)

# Wrap SVR in MultiOutputRegressor
multi_output_svr = MultiOutputRegressor(svr_model)

# Train the model
multi_output_svr.fit(x_train, y_train)

# Make predictions
y_pred = multi_output_svr.predict(x_test)

# Evaluate the model
# mae = mean_absolute_error(y_test, y_pred)
# mse = mean_squared_error(y_test, y_pred)
# rmse = np.sqrt(mse)
# r2 = r2_score(y_test, y_pred)

# print("Evaluation Metrics:")
# print(f"Mean Absolute Error (MAE): {mae:.4f}")
# print(f"Mean Squared Error (MSE): {mse:.4f}")
# print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
# print(f"R2 Score: {r2:.4f}")
# Evaluate the model
mae = mean_absolute_error(y_test, y_pred, multioutput='raw_values')  # Evaluate for each output

print("Mean Absolute Error for each output:")
print(mae)


Mean Absolute Error for each output:
[6.7166584  2.52376129 0.3891885 ]


GAT

In [71]:
# # Create graph data for each floor
# def create_graphs_by_floor(data, sensor_cols):
#     graphs = {}
#     for floor_id in data['Floor_ID'].unique():
#         floor_data = data[data['Floor_ID'] == floor_id]
#         x = torch.tensor(floor_data[sensor_cols].values, dtype=torch.float)
#         y = torch.tensor(floor_data[['Pos_x', 'Pos_y']].values, dtype=torch.float)

#         # Create dummy edges (fully connected graph for simplicity)
#         edge_index = torch.combinations(torch.arange(x.size(0)), r=2).T
#         edge_index = torch.cat([edge_index, edge_index.flip(0)], dim=1)

#         graphs[floor_id] = Data(x=x, edge_index=edge_index, y=y)
#     return graphs

# graphs_by_floor = create_graphs_by_floor(df_train, sensor_cols)

from sklearn.metrics.pairwise import euclidean_distances
import numpy as np

def create_knn_graphs_by_floor(data, sensor_cols, k=5):
    graphs = {}
    for floor_id in data['Floor_ID'].unique():
        floor_data = data[data['Floor_ID'] == floor_id]
        x = torch.tensor(floor_data[sensor_cols].values, dtype=torch.float)
        y = torch.tensor(floor_data[['Pos_x', 'Pos_y', 'Floor_ID']].values, dtype=torch.float)

        # Compute pairwise distances
        distances = euclidean_distances(floor_data[sensor_cols])
        edge_index = []

        # For each node, find k nearest neighbors
        for i in range(distances.shape[0]):
            neighbors = np.argsort(distances[i])[:k]  # Get indices of k nearest neighbors
            for j in neighbors:
                edge_index.append([i, j])

        # Convert edge list to PyTorch Geometric format
        edge_index = torch.tensor(edge_index, dtype=torch.long).T

        graphs[floor_id] = Data(x=x, edge_index=edge_index, y=y)
    return graphs

graphs_by_floor = create_knn_graphs_by_floor(df_train, sensor_cols, k=5)


In [72]:
# class GraphSAGE(torch.nn.Module):
#     def __init__(self, in_channels, hidden_channels, out_channels):
#         super(GraphSAGE, self).__init__()
#         self.conv1 = SAGEConv(in_channels, hidden_channels)
#         self.conv2 = SAGEConv(hidden_channels, out_channels)

#     def forward(self, x, edge_index):
#         x = self.conv1(x, edge_index).relu()
#         x = self.conv2(x, edge_index)
#         return x

from torch_geometric.nn import GATConv

class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads):
        super(GAT, self).__init__()
        self.conv1 = GATConv(in_channels, hidden_channels, heads=heads, concat=True)
        self.conv2 = GATConv(hidden_channels * heads, out_channels, heads=1, concat=False)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Initialize the GAT model
model = GAT(
    in_channels=len(sensor_cols),
    hidden_channels=64,
    out_channels=3,
    heads=3  # Multi-head attention for the first layer
).to(device)


In [73]:
# # Initialize and train the model for one floor
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = GraphSAGE(in_channels=len(sensor_cols), hidden_channels=64, out_channels=2).to(device)

# # Train on a single floor and fine-tune on another
# def train_model(graph_data, model, epochs=100, lr=0.01):
#     graph_data = graph_data.to(device)
#     optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#     criterion = torch.nn.MSELoss()

#     for epoch in range(epochs):
#         model.train()
#         optimizer.zero_grad()
#         pred = model(graph_data.x, graph_data.edge_index)
#         loss = criterion(pred, graph_data.y)
#         loss.backward()
#         optimizer.step()
#         if (epoch + 1) % 20 == 0:
#             print(f'Epoch {epoch+1}, Loss: {loss.item()}')
#     return model

def train_model(graph_data, model, epochs=200, lr=0.01):
    graph_data = graph_data.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.MSELoss()

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        pred = model(graph_data.x, graph_data.edge_index)
        loss = criterion(pred, graph_data.y)
        loss.backward()
        optimizer.step()
        if (epoch + 1) % 20 == 0:
            print(f'Epoch {epoch+1}, Loss: {loss.item()}')
    return model

In [74]:
max_floor = 0
for floor_id in sorted(graphs_by_floor.keys()):
    print(f"Training/Fine-tuning on Floor {floor_id}...")
    model = train_model(graphs_by_floor[floor_id], model)
    max_floor = floor_id

Training/Fine-tuning on Floor -3...
Epoch 20, Loss: 4391.08642578125
Epoch 40, Loss: 558.4625854492188
Epoch 60, Loss: 25.355018615722656
Epoch 80, Loss: 21.469118118286133
Epoch 100, Loss: 12.596022605895996
Epoch 120, Loss: 10.984979629516602
Epoch 140, Loss: 9.953471183776855
Epoch 160, Loss: 9.176854133605957
Epoch 180, Loss: 8.527734756469727
Epoch 200, Loss: 7.931508541107178
Training/Fine-tuning on Floor -2...
Epoch 20, Loss: 67.92457580566406
Epoch 40, Loss: 78.45779418945312
Epoch 60, Loss: 31.839820861816406
Epoch 80, Loss: 12.57603931427002
Epoch 100, Loss: 9.01495361328125
Epoch 120, Loss: 7.941660404205322
Epoch 140, Loss: 7.298122882843018
Epoch 160, Loss: 6.769341468811035
Epoch 180, Loss: 6.304450511932373
Epoch 200, Loss: 5.884345531463623
Training/Fine-tuning on Floor -1...
Epoch 20, Loss: 187.65869140625
Epoch 40, Loss: 86.3884048461914
Epoch 60, Loss: 26.526016235351562
Epoch 80, Loss: 15.775426864624023
Epoch 100, Loss: 13.002857208251953
Epoch 120, Loss: 11.466488

In [75]:
# from sklearn.metrics import r2_score

# # Evaluate the global model on a new dataset
# def evaluate_model(model, test_graph):
#     model.eval()
#     test_graph = test_graph.to(device)
#     with torch.no_grad():
#         pred = model(test_graph.x, test_graph.edge_index)
#         true = test_graph.y

#         # Calculate Mean Absolute Error (MAE)
#         mae = torch.mean(torch.abs(pred - true))

#         # Calculate Root Mean Square Error (RMSE)
#         rmse = torch.sqrt(torch.mean((pred - true) ** 2))

#         r2 = r2_score(true.cpu().numpy(), pred.cpu().numpy())

#     print("Evaluation Results:")
#     print(f"MAE: {mae.item():.4f}")
#     print(f"RMSE: {rmse.item():.4f}")
#     print(f"R2 Score: {r2:.4f}")
#     return mae.item(), rmse.item()


from sklearn.metrics import r2_score

# Evaluate the global model on a new dataset
def evaluate_model(model, test_graph):
    model.eval()
    test_graph = test_graph.to(device)
    with torch.no_grad():
        pred = model(test_graph.x, test_graph.edge_index)  # Predict positions
        predx = pred[:, :1]
        predy = pred[:, 1:2]
        true = test_graph.y  # True positions
        truex = true[:, :1]
        truey = true[:, 1:2]


        # Calculate Mean Absolute Error (MAE)
        maex = torch.mean(torch.abs(predx - truex))

        # Calculate Root Mean Square Error (RMSE)
        rmsex = torch.sqrt(torch.mean((predx - truex) ** 2))

        # Calculate Mean Absolute Percentage Error (MAPE)
        mapex = torch.mean(torch.abs((predx - truex) / truex)) * 100

        # Calculate R2 Score
        r2x = r2_score(truex.cpu().numpy(), predx.cpu().numpy())

        # Calculate Mean Absolute Error (MAE)
        maey = torch.mean(torch.abs(predy - truey))

        # Calculate Root Mean Square Error (RMSE)
        rmsey = torch.sqrt(torch.mean((predy - truey) ** 2))

        # Calculate Mean Absolute Percentage Error (MAPE)
        mapey = torch.mean(torch.abs((predy - truey) / truey)) * 100

        # Calculate R2 Score
        r2y = r2_score(truey.cpu().numpy(), predy.cpu().numpy())

    print("Evaluation Results X:")
    print(f"MAE: {maex.item():.4f}") # Probably main evaluator
    print(f"RMSE: {rmsex.item():.4f}")
    print(f"MAPE: {mapex.item():.4f}%")
    # print(f"R2 Score: {r2x:.4f}") # not effective because data not in a normal distribution

    print("Evaluation Results Y:")
    print(f"MAE: {maey.item():.4f}") # Probably main evaluator
    print(f"RMSE: {rmsey.item():.4f}")
    print(f"MAPE: {mapey.item():.4f}%")
    # print(f"R2 Score: {r2y:.4f}") # not effective because data not in a normal distribution


# # Create a unified graph without floor consideration
# def create_unified_graph(data, sensor_cols):
#     # Features: RSSI values
#     x = torch.tensor(data[sensor_cols].values, dtype=torch.float)
#     # Labels: Coordinates (Pos_x, Pos_y)
#     y = torch.tensor(data[['Pos_x', 'Pos_y']].values, dtype=torch.float)

#     # Create a fully connected graph for simplicity
#     num_nodes = x.size(0)
#     edge_index = torch.combinations(torch.arange(num_nodes), r=2).T
#     edge_index = torch.cat([edge_index, edge_index.flip(0)], dim=1)  # Make edges bidirectional

#     return Data(x=x, edge_index=edge_index, y=y)

def create_knn_unified_graph(data, sensor_cols, k=5):
    # Features: RSSI values
    x = torch.tensor(data[sensor_cols].values, dtype=torch.float)
    # Labels: Coordinates (Pos_x, Pos_y)
    y = torch.tensor(data[['Pos_x', 'Pos_y', 'Floor_ID']].values, dtype=torch.float)

    # Compute pairwise distances
    from sklearn.metrics.pairwise import euclidean_distances
    distances = euclidean_distances(data[sensor_cols])
    edge_index = []

    # Construct edges using KNN
    for i in range(distances.shape[0]):
        neighbors = np.argsort(distances[i])[:k]  # Get k nearest neighbors
        for j in neighbors:
            edge_index.append([i, j])

    # Convert edge list to PyTorch Geometric format
    edge_index = torch.tensor(edge_index, dtype=torch.long).T

    return Data(x=x, edge_index=edge_index, y=y)


# Create graph for new dataset
# new_global_graph = create_unified_graph(df_test, sensor_cols)
new_global_graph = create_knn_unified_graph(df_test, sensor_cols, k=5)

# Evaluate the global model on the new graph
print("Evaluating on the new dataset...")
evaluate_model(model, new_global_graph)

Evaluating on the new dataset...
Evaluation Results X:
MAE: 16.2252
RMSE: 19.6578
MAPE: 33.4324%
Evaluation Results Y:
MAE: 4.0980
RMSE: 5.4321
MAPE: 35.3372%


In [78]:
def train_model_final(graph_data, model, epochs=200, lr=0.01):
    graph_data = graph_data.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.MSELoss()

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        pred = model(graph_data.x, graph_data.edge_index)
        loss = criterion(pred, graph_data.y)
        loss.backward()
        optimizer.step()
        if (epoch + 1) % 20 == 0:
            print(f'Epoch {epoch+1}, Loss: {loss.item()}')
    return model

final_train_graph = create_knn_graphs_by_floor(df_train, sensor_cols, k=5)
test_graph = create_knn_graphs_by_floor(df_test, sensor_cols, k=5)
curr_floor = 0
for floor_id in sorted(final_train_graph.keys()):
    print(f"Training/Fine-tuning on Floor {floor_id}...")
    model_final = train_model_final(final_train_graph[floor_id], model)
    curr_floor = floor_id
    print("Evaluating model on floor " + str(floor_id) + "...")
    evaluate_model(model, test_graph[floor_id])

Training/Fine-tuning on Floor -3...
Epoch 20, Loss: 8.517341613769531
Epoch 40, Loss: 7.207226276397705
Epoch 60, Loss: 5.008084774017334
Epoch 80, Loss: 4.564362525939941
Epoch 100, Loss: 4.387905120849609
Epoch 120, Loss: 4.278393268585205
Epoch 140, Loss: 4.198227405548096
Epoch 160, Loss: 4.134030818939209
Epoch 180, Loss: 4.080323696136475
Epoch 200, Loss: 4.034214973449707
Evaluating model on floor -3...
Evaluation Results X:
MAE: 9.7613
RMSE: 11.2880
MAPE: 12.4914%
Evaluation Results Y:
MAE: 2.1046
RMSE: 2.5900
MAPE: 10.2582%
Training/Fine-tuning on Floor -2...
Epoch 20, Loss: 30.67874526977539
Epoch 40, Loss: 11.752347946166992
Epoch 60, Loss: 8.74910831451416
Epoch 80, Loss: 8.150623321533203
Epoch 100, Loss: 7.69141149520874
Epoch 120, Loss: 7.40309476852417
Epoch 140, Loss: 7.16849946975708
Epoch 160, Loss: 6.972365379333496
Epoch 180, Loss: 6.807369709014893
Epoch 200, Loss: 6.667547702789307
Evaluating model on floor -2...
Evaluation Results X:
MAE: 12.3591
RMSE: 14.6304
M

In [77]:
# from torch_geometric.data import DataLoader

# # Loop through each floor's dataset
# models_per_floor = {}
# for floor_id, floor_data in floor_datasets.items():
#     # Preprocess data for the floor (convert to graph)
#     graph_data = preprocess_graph(floor_data)  # Define preprocessing logic

#     # Train a GraphSAGE model
#     model = GraphSAGEModel(input_dim=graph_data.num_node_features, hidden_dim=64, output_dim=2)
#     optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

#     for epoch in range(epochs):
#         model.train()
#         optimizer.zero_grad()
#         coords_pred, floor_pred = model(graph_data)
#         loss = nn.MSELoss()(coords_pred, graph_data.y[:, :2])  # Only use (Pos_x, Pos_y)
#         loss.backward()
#         optimizer.step()

#     # Save trained model for the floor
#     models_per_floor[floor_id] = model